<a href="https://colab.research.google.com/github/morettitommaso/sql-exercises/blob/main/edb02_librerie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Istruzioni per iniziare
Per **lavorare** su questo [jupyter notebook](https://jupyter-notebook.readthedocs.io/en/stable/notebook.html) **devi**

1. **Salva il notebook** sul tuo Google Drive. Fino a che non salvi il notebook, non potrai né modificarlo né interagire.

  * Seleziona **File** / **Save a copy in Drive** dal menù Colab (qua sotto un esempio del menù Colab)
     ![Colab Menu](https://drive.google.com/uc?export=download&id=1-WfIFWuHC6OSJb3iwnR7NqpkXs9tvwO2)
  * Se richiesto, effettua l'autenticazione con un tuo account google
  ![Signin Button](https://drive.google.com/uc?export=download&id=1yomWF3t03TiPsrp6AAZDXIFpz5XXTvM1) (puoi usare le credenziali campus oppure un tuo account gmail personale)
  

2.  I file vengono normalmente salvati in una *directory* **Colaboratory** creata sotto la directory principale del vostro google drive.

3. Per aprire il notebook nuovamente:
    * accedi al google drive dove hai salvato il notebook [drive.google.com](http://drive.google.com/)
    * apri la directory **Colaboratory**
    * click con il tasto **destro** del mouse sul file. Dal menù scegli **Open With** / **Google Colaboratory**

4. Se avete problemi, consultate [questo documento](https://docs.google.com/document/d/1Y-ABvbOQhMvi7COibLJopL-mnPsaCBv_KRr6eKAsnj8/edit?usp=sharing)




# Come usare un Jupyter Notebook

Questo è un [jupyter notebook](https://jupyter-notebook.readthedocs.io/en/stable/notebook.html).

Un notebook può contenere diversi tipi di celle.

* Le [Markdown cells](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html) contengono commenti scritti usando una
[sintassi](https://en.support.wordpress.com/markdown-quick-reference/) specifica
* Le Code cells possono contenere codice python, query SQL, ...

## Esecuzione di una cella
Per eseguire il codice contenuto in una cella potete (a vostra scelta)
* Premere il tasto play ![Play Button](https://drive.google.com/uc?export=download&id=11d9Z5aBUWqwskHLy2KKWwEFdO4LjFSpJ) a lato della cella (visibile solo dopo aver fatto un click sulla cella stessa)

* Fare un click per selezionare la cella e premere **SHIFT Invio** da tastiera
* Utilizzando uno dei comandi presenti nel menù *Runtime*


## Combinazioni di tasti
Alcune combinazioni di tasti usate frequentemente (al posto del tasto *Ctrl* usate il tasto *Command* nel Mac)

* Ctrl+M  M      (per trasformare la cella corrente in una cella *Markdown*)
* Ctrl+M  Y      (per trasformare la cella corrente in una cella *codice*)
* Ctrl+M L       (fa apparire/sparire i numeri di riga)

L'elenco completo dei comandi può essere visualizzato tramite la voce del Menù Colab: Tools / Keyboard Shortcuts


# Caricamento dati nel DBMS

La cella qua sotto serve per per scaricare il database e interagire con il DBMS.

Se il DBMS semtte di funzionare, rieseguite la  cella di codice sottostante.

In [ ]:
# Per favore, non modificate il codice di questa cella
urldb='https://drive.google.com/uc?export=download&id=1wS2sFxQlw3SycoqNfY5lnSyH0HRYL7i-'
fn='db02librerie.db'
pwd=''

import sqlite3
import urllib
import urllib.request
import zipfile
import os

import pandas as pd
pd.set_option('display.max_colwidth', None) # -1
pd.set_option('display.max_rows', 100)
from IPython.display import display, HTML

def downloadDBFile(urlFile, localFileName, pwd=''):
  # Dowloading the .zip file into the colab remote computation facility

  print('Downloading db file')
  try:
    urllib.urlretrieve(urlFile, localFileName) # Works in Python2
    print('Python2 detected')
  except AttributeError:
    print('Python 3 detected')
    with urllib.request.urlopen(urlFile) as response: # Works in Python3
      data = response.read()
      fd=open(localFileName,'wb')
      fd.write(data)
    fd.close()
  print('Download Done')
  if localFileName.endswith('.zip'):
    zipFileName = localFileName
    localFileName = zipFileName[:-4]
    print('Unzipping the file content')
    with zipfile.ZipFile(zipFileName, 'r') as zip_ref:
      zip_ref.extractall(localFileName)
      print('File %s extracted' % localFileName)
  elif localFileName.endswith('.pez'):
    zipFileName = localFileName
    localFileName = zipFileName[:-4]
    print('Unzipping the password protected file content')
    with zipfile.ZipFile(zipFileName, 'r') as zip_ref:
      #zip_ref.extractall(localFileName, pwd=bytes(pwd,'utf-8')) # it messes up the paths with password encrypted files
      zip_ref.extract(localFileName, pwd=bytes(pwd,'utf-8')) # localFileName: the name of the file in the .zip archive
      print('File %s extracted' % localFileName)

  localFileName = os.path.abspath(localFileName)
  print('File saved in '+localFileName)
  return localFileName
  print('Unzipping the password protected file content')


dbFileFullPath = downloadDBFile(urldb, fn, pwd=pwd)

def rq(query, threshold=100): # rq is for Run Query
    conn = sqlite3.connect(dbFileFullPath)
    if query.replace(' ','').replace('\n','') == '':
      return
    try:
        res = pd.read_sql(query,conn)
        conn.close()
        nrecords=res.shape[0]
        if nrecords<=threshold:
          display(res)
          print("%d records returned" % (nrecords) )
          #HTML("<b> %d records returned </b>" % (nrecords) )
          #return res

        else:
          display(res.iloc[:threshold,:])
          print("Too many records in the result, returned only the first %d" % (threshold))
          #return res.iloc[:threshold,:]

    except (pd.io.sql.DatabaseError, Exception)  as dbe: # (sqlite3.DatabaseError , sqlite3.OperationalError)
        print('Error')
        print(type(dbe))
        print(dbe)

Python 3 detected
Download Done
File saved in /content/db02librerie.db


# EDB02 librerie

In questo esercizio dovrete interagire con un database di nome *librerie* composto da diverse tabelle.

## Schema ER
Qua sotto è riportato lo schema ER del database librerie.

<img src="https://drive.google.com/uc?export=download&id=1JCgOKP3WwSQn7xmBB1DQN1c61zjx59M3" alt="Database ER Schema" width="500">
<!--
https://drive.google.com/file/d/1JCgOKP3WwSQn7xmBB1DQN1c61zjx59M3/view?usp=sharing
-->

[Link](https://drive.google.com/file/d/1JCgOKP3WwSQn7xmBB1DQN1c61zjx59M3/view?usp=sharing) per aprire lo schema in un'altra finestra.

## Schema logico relazionale
Nella figura qua sotto è riportato lo schema relazionale del DB librerie (nota bene: si tratta di uno schema relazionale, non di uno schema Entità Relazioni).


<img src="https://drive.google.com/uc?export=download&id=1yQNp8zqL1423r33lIvSpV0lxASt5UBrr" alt="Database Schema" width="600">
<!-- https://www.dropbox.com/s/r115zp5k2k2zv8l/db02librerieSchema.png?dl=1 -->

Agli estremi di ogni freccia, sono presenti 2 simboli (uno per la cardinalità minima, uno per la cardinalità massima), ogni simbolo può essere un elemento del seguente alfabeto (tra parentesi i significati):

* O (zero)
* | (uno)
* <img src="https://docs.google.com/drawings/d/e/2PACX-1vRbanf6cNiZWPXTDatkH-xNadoI8ZKGuLq2au4GYB1QkrXH528yClegyf5G1pFU6vy5TnlPKQ1ti1dj/pub?w=360&amp;h=20"><!-- &#5833; --> (molti)

Es. <img src="https://docs.google.com/drawings/d/e/2PACX-1vTYY7uTwq8rVX7ZhZJnKerDeICjkEkPWnJDacJO7XnAUZjGQEapRHxjqiKr8ui4QHudPZtQacXI7aCm/pub?w=41&amp;h=27"> viene interpretato come [1:N] (minima 1, massima N).

Il modo di leggere le numerosità in questi schemi relazionali è invertita rispetto agli schemi ER.

[Link](https://drive.google.com/file/d/1yQNp8zqL1423r33lIvSpV0lxASt5UBrr/view?usp=sharing) per aprire lo schema in un'altra finestra.



# Esercizi

Svolgete gli esercizi indicati nelle celle sottostanti.

**Riepilogo sintassi query SQL**

SELECT Attributo1, Attributo2, ..., *

FROM Tabella1, Tabella2, …, TabellaN

WHERE CondizioniSemplici

ORDER BY ListaAttributiDiOrdinamento

;


## (EDB02) 1

Ispezionate le tabelle del database librerie e cercate di farvi un’idea del contenuto del database.

Per ispezionare una tabella, potete utilizzare la query generica:

SELECT * FROM  scrivi_qui_il_nome_della_tabella;

# Per i nomi delle tabelle fate riferimento allo schema della figura precedente.


In [ ]:
rq("""
select * from autori;
""")

# Potete aggiungere altre caselle di codice qua sotto per svolgere altre query

,codice_autore,cognome,nome
0,1,Archer,Jeffrey
1,2,Christie,Agatha
2,3,Clarke,Arthur C.
3,4,Francis,Dick
4,5,Cussler,Clive
5,6,King,Stephen
6,7,Pratt,Philip
7,8,Adamski,Joseph
8,10,Harmon,Willis
9,11,Rheingold,Howard


22 records returned


In [ ]:
rq("""

""")


## (EDB02) 2

Scrivete su un foglio di carta il nome delle tabelle principali, per ogni tabella individuate (se presenti):

* la chiave primaria (o le chiavi primarie se più attributi sono coinvolti)

* i collegamenti con le altre tabelle (individuate cioè le chiavi secondarie). Suggerimento: tracciate delle frecce per collegare le chiavi esterne con le corrispondenti chiavi primarie

* nello schema sono presenti 2 tabelle ponte. Quali sono? (Se non sapete che cosa è una tabella ponte, ignorate questo punto)

Questo foglio sarà utilizzato negli esercizi successivi.

NB: scrivere le tabelle e le chiavi su carta, vi sarà molto utile come riferimento per svolgere gli esercizi indicati di seguito, dedicate un tempo congruo ma senza esagerare per svolgere questo punto (NON più di 10 minuti).


## (EDB02) 3

Visualizzate il contenuto della tabella libri

In [ ]:
rq("""

""")

## (EDB02) 4

Visualizzate solo il contenuto degli attributi titolo, tipo, prezzo della tabella libri.

In [ ]:
rq("""

""")

## (EDB02) 5

Visualizzate titolo e prezzo dei libri che costano meno di 10 euro.

Nota: 17 tuple nel risultato

In [ ]:
rq("""

""")

## (EDB02) 6

Ora estrarremo alcune informazioni sulle tipologie di libri.

#### (EDB02) 6.a

Visualizzate le informazioni dei libri di informatica (i libri con tipo uguale a 'CS'), CS è l'acronimo di "computer science".

Nota: 4 tuple nel risultato


In [ ]:
rq("""

""")

#### (EDB02) 6.b

Visualizzate le informazioni dei libri di *science fiction* (i libri con tipo uguale a 'FIC'), FIC è usato per indicare *Science Fiction*.

Nota: 6 tuple nel risultato

In [ ]:
rq("""

""")

#### (EDB02) 6.c

Visualizzate con un’unica query le informazioni sia dei libri di *informatica* sia dei libri di *science fiction* (tipi: rispettivamente 'CS' e 'FIC').

Nota: 10 tuple di risultato.


In [ ]:
rq("""

""")

#### (EDB02) 6.d

Se nella query precedente avete usato l’operatore OR, provate ad usare l’operatore AND. Se invece avete usato l’AND provate con l’OR. Cosa cambia? Qual è il motivo? Suggerimento: se a questo punto avete dei dubbi (anche minimi), chiamate il docente/tutor che trovate in aula.

In [ ]:
rq("""

""")

## (EDB02) 7

Ora andremo a scrivere delle query in cui dovremo mettere in join più tabelle.

Per svolgere questo esercizio dovete aver sottomano il foglio che avete annotato/creato nell'esercizio 2.

Se vi sentite molto ferrati sull’argomento potete saltare direttamente al *sottopunto g*, altrimenti svolgete tutti i sottopunti seguenti.

#### (EDB02) 7.a

Con una query, visualizzate dalla tabella libri i dati del libro dal titolo "Shyness". A seconda del DBMS che state usando, il contenuto delle stringhe può essere *case sensitive* oppure no. I comandi SQL sono invece sempre *case insensitive*.

Nota: 1 record nel risultato.

In [ ]:
rq("""

""")

#### (EDB02) 7.b

Nella tabella libri, qual è la *foreign key* che  permette di collegare un libro al suo editore?

Per il libro dal titolo Shyness, qual è il valore assunto dalla foreign key?

Scrivete la vostra risposta in questa cella (doppio click per entrare in modalità editing)

#### (EDB02) 7.c

(Con una query) visualizzate il contenuto della tabella editori.

In [ ]:
rq("""

""")

#### (EDB02) 7.d

Rispondete alle prossime 2 domande guardando il risultato della query precedente (non dovete scrivere altre query).

Qual è l'attributo che funge da chiave della tabella editori (a cui fa riferimento la chiave straniera presente nella tabella libri)?

Qual è l'editore che edita il libro Shyness?

Scrivete la vostra risposta in questa cella (doppio click per entrare in modalità editing)


#### (EDB02) 7.e

Eseguite la query

select * from libri, editori;

In [ ]:
rq("""

""")

#### (EDB02) 7.f

Nel risultato della query precedente, guardate i record che fanno riferimento al libro Shyness *Shyness*. Il libro è stato associato ad editori in maniera errata? (Non serve scrivere la risposta).

Vi ricordate come dal prodotto cartesiano si passa ad un join? Se non ve lo ricordate, andate a ripassare la parte di teoria corrispondente.


#### (EDB02) 7.g

Visualizzate per ogni libro, il titolo del libro e il nome dell'editore che lo edita (dovete effettuare il join tra la tabella libri e la tabella editori).

Nota: 29 tuple nel risultato

In [ ]:
rq("""

""")

## (EDB02) 8

Con una query, visualizzate per ogni libro il titolo, il prezzo e la descrizione del formato (vedi tabella formato).

Nota: 29 tuple nel risultato

In [ ]:
rq("""

""")

## (EDB02) 9

Ora ci occuperemo degli autori dei libri

#### (EDB02) 9.a

Per visualizzare con un'unica query il titolo del libro e il cognome dell'autore che lo ha scritto, quante tabelle dovete mettere in join? Quali tabelle? Individuate le *foreign keys* e le chiavi (primarie) a cui fanno riferimento.

Scrivete la vostra risposta in questa cella (doppio click per entrare in modalità editing)

#### (EDB02) 9.b

Visualizzate con un’unica query il titolo del libro e il cognome dell'autore che lo ha scritto.

Nota: 33 tuple nel risultato.

In [ ]:
rq("""

""")

#### (EDB02) 9.c

Nella query precedente, secondo voi, perché ci sono 33 tuple nel risultato e non 29 (il numero di record presenti nella tabella libri)?

Scrivete la vostra risposta in questa cella (doppio click per entrare in modalità editing)

## (EDB02) 10

Come il punto precedente (... il titolo del libro e il cognome dell'autore che lo ha scritto), ma visualizzate solamente i dati dei libri scritti da Kafka.

Nota: 2 tuple nel risultato

In [ ]:
rq("""

""")

## (EDB02) 11

Come il punto precedente (... il titolo del libro e il cognome dell'autore che lo ha scritto), ma visualizzate solamente i libri scritti da Kafka o da Agata Cristie.

Nota: 4 tuple nel risultato

ps: controllate nel database come sono stati scritti i nomi e cognomi dei due autori :-)

In [ ]:
rq("""

""")

## (EDB02) 12
Visualizzate il nome e la sede delle librerie dove è in vendita il libro 'dBASE Programming'.

Nota: 1 tupla nel risultato

In [ ]:
rq("""

""")

(EDB02) 13

Svolgete delle interrogazioni a piacere

In [ ]:
rq("""

""")
# aggiungete altre celle di codice qua sotto, se necessario

# Congratulazioni

Hai terminato EDB02

Ora vorrei chiederti di compilare un breve questionario di valutazione dell'esericizio che trovi presso [questo link](https://docs.google.com/forms/d/e/1FAIpQLSfJhbGtuOJma5oMdvjKnifAzY958RklbQClBAFC0Zqt1aBKig/viewform?usp=sf_link).

Le informazioni fornite saranno utili per migliorare questo e i prossimi esercizi.

Grazie in anticipo

Se vuoi proporre al docente uno o più punti da inserire in questo esercizio, mandagli una email specificando: il testo della domanda (inclusi i suggerimenti per controllare se la risposta è esatta) e la query da svolgere.

Mirko Cesarini